In [ ]:
from functools import reduce
from pathlib import Path
from typing import List

from itertools import chain

import json
import yaml

In [ ]:
hmd_folder = Path('/') / 'hmd'
project_name = 'demo-notebooks-idl'

project_folder = hmd_folder / 'projects' / project_name

idl_folder = project_folder / 'src' / 'idl'

ts_folder = project_folder / 'src' / "tabular_structures"

idl_file = idl_folder / 'device.yaml'
print('variables setup')

In [ ]:
with idl_file.open('r') as idl_handle:
    idl_data = yaml.safe_load(idl_handle)
print('idl loaded')

In [ ]:
def build_structure(name, add_prefix=True):
    prefix = None
    if add_prefix:
        prefix = name
    struct = idl_data['structures'][name]
    return list(map(lambda kv: build_column(kv[0], kv[1], prefix=prefix), struct['columns'].items()))

def build_column(column_name, column_type, prefix=None):
    column_segments = column_type.split(':')
    full_column_name = column_name
    if prefix:
        full_column_name = f"{prefix}.{column_name}"
    result =  {
      "identifier": f"tab_col-{full_column_name}",
      "order": 0,
      "definition": {
        "__schema": "hmd_lang_structure.tabular_column",
        "name": full_column_name,
        "datatype": column_type,
        "identifier": f"tab_col_def-{full_column_name}"
      }
    }
    if column_segments[0] == 'structure':
        result = build_structure(column_segments[1])

    return result

def build_type(name, value):
    columns = []
    for column_name, column_type in value['columns'].items():
        column_segments = column_type.split(':')
        if column_segments[0] == 'structure':
            columns = [*columns, *build_structure(column_segments[1])]
        else:
            columns.append(build_column(column_name, column_type))
    if 'extends' in value:
        ext_columns = build_structure(value['extends'], add_prefix=False)
        columns = chain(columns, ext_columns)
    result = {
        "__schema": "hmd_lang_structure.tabular_structure",
        "name": name,
        "identifier": f"tab_struct-{name}",
        "columns": list(columns)
    }
    return result
print('helpers defined')

In [ ]:
ts_folder.mkdir(parents=True, exist_ok=True)
for name, value in idl_data['types'].items():
    ts_file = ts_folder / f'{name}.hmdstruct'
    with ts_file.open('w') as ts_handle:
        json.dump(build_type(name, value), ts_handle, indent=2)
        print(f"Wrote to {ts_file}")